In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
from functools import partial

import os
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings('ignore')
AUTO = tf.data.experimental.AUTOTUNE


In [2]:
# # Extract zip file downloaded from kaggle
# from zipfile import ZipFile
# data_path = r'C:\Users\haris\AI\ML\Projects\Dog Breed Classification\archive.zip'

# with ZipFile(data_path, 'r') as zip:
# 	zip.extractall()
# 	print('The data set has been extracted.')

In [3]:
# Path to the folders containing images and annotations
annotations_folder = r'C:\Users\haris\AI\ML\Projects\Dog Breed Classification\annotations'
images_folder = r'C:\Users\haris\AI\ML\Projects\Dog Breed Classification\images'

# Initialize lists to store image data and labels
images = []
labels = []

# Parse XML annotations and load images
for breed_folder in os.listdir(annotations_folder):
    breed_path = os.path.join(annotations_folder, breed_folder)
    for annotation_file in os.listdir(breed_path):
        annotation_path = os.path.join(breed_path, annotation_file)
        
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        
        breed = root.find('object/name').text
        xmin = int(root.find('object/bndbox/xmin').text)
        ymin = int(root.find('object/bndbox/ymin').text)
        xmax = int(root.find('object/bndbox/xmax').text)
        ymax = int(root.find('object/bndbox/ymax').text)
        
        img_path = os.path.join(images_folder, breed_folder, annotation_file + '.jpg')
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224, 224))  # Resize to desired size
        
        images.append(image)
        labels.append(breed)

# Convert labels to numerical format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [4]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert lists to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Print dataset shapes for verification
print("Train data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Test data shape:", X_test.shape)
print("Train labels shape:", y_train.shape)
print("Validation labels shape:", y_val.shape)
print("Test labels shape:", y_test.shape)


Train data shape: (14406, 224, 224, 3)
Validation data shape: (3087, 224, 224, 3)
Test data shape: (3087, 224, 224, 3)
Train labels shape: (14406,)
Validation labels shape: (3087,)
Test labels shape: (3087,)


In [5]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load the Xception model pretrained on ImageNet
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(120, activation='softmax')(x)  # 120 classes for dog breeds

# Create the new model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

83683744/83683744 [==============================] - 24s 0us/step
Epoch 1/10
451/451 [==============================] - 65s 89ms/step - loss: 5.0801 - accuracy: 0.0117 - val_loss: 4.8085 - val_accuracy: 0.0078
Epoch 2/10
451/451 [==============================] - 38s 84ms/step - loss: 4.7802 - accuracy: 0.0146 - val_loss: 4.7928 - val_accuracy: 0.0091
Epoch 3/10
451/451 [==============================] - 38s 84ms/step - loss: 4.7618 - accuracy: 0.0161 - val_loss: 4.7863 - val_accuracy: 0.0100
Epoch 4/10
451/451 [==============================] - 38s 85ms/step - loss: 4.7424 - accuracy: 0.0185 - val_loss: 4.7888 - val_accuracy: 0.0097
Epoch 5/10
451/451 [==============================] - 38s 85ms/step - loss: 4.7171 - accuracy: 0.0204 - val_loss: 4.7979 - val_accuracy: 0.0136
Epoch 6/10
451/451 [==============================] - 39s 86ms/step - loss: 4.6936 - accuracy: 0.0217 - val_loss: 4.7621 - val_accuracy: 0.0136
Epoch 7/10
451/451 [==============================] - 39s 86ms/step - 

ResourceExhaustedError: Graph execution error:

Detected at node 'model/block2_sepconv1/separable_conv2d' defined at (most recent call last):
    File "C:\Users\haris\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\haris\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\haris\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\haris\anaconda3\envs\tensorflow-gpu\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\haris\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\haris\anaconda3\envs\tensorflow-gpu\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.as